# 모델링을 위한 질문 생성 코드

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# %pip install nest_asyncio

In [ ]:
import os
import asyncio
import json
import random
import nest_asyncio
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 1. 주피터 노트북 환경 설정
nest_asyncio.apply()

# 2. 모델 및 파일 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
OUTPUT_FILE = 'final_dataset.json'
sem = asyncio.Semaphore(15)  # 동시 요청 제한

# 3. 현실적인 페르소나 정의
PERSONA_EXAMPLES = """
[현실적인 질문 문체 예시]
1. 취준생: "매일 지하철로 학원 다니느라 대중교통비가 너무 부담돼요. 편의점 도시락도 자주 사 먹는데 이런 혜택 위주인 카드가 있을까요?"
2. 직장인: "차로 출퇴근해서 주유 할인이 꼭 필요해요. 가끔 병원이나 약국 갈 때도 혜택을 볼 수 있는 카드가 있는지 궁금합니다."
3. 학부모: "주말마다 마트에서 장을 보는데 할인율이 높았으면 좋겠어요. 아이 학원비 결제할 때도 도움이 되는 카드가 있을까요?"
"""

# 4. 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", f"당신은 금융 데이터 전문가입니다. 사용자가 모바일 앱에서 문의할 법한 자연스러운 문장을 생성하세요.\n{PERSONA_EXAMPLES}"),
    ("user", """아래 [혜택 정보]를 보고 질문과 '요약된 정답' 쌍을 생성하세요.

[혜택 정보]: {benefit_content}

조건:
1. anchor: 제시된 예시를 참고하여 **100~150자 내외**의 1인칭 질문을 생성하세요. 카드사 이름은 언급하지 마세요.
2. positive: 핵심 혜택과 전월실적 조건만 추출하여 150~200자 내외로 간결하게 요약하세요.
3. 결과는 반드시 아래 JSON 형식으로만 답변하세요.
{{
    "anchor": "100~150자의 현실적인 질문",
    "positive": "요약된 핵심 혜택"
}}
""")
])

# 5. 재시도 로직 및 세마포어
async def invoke_with_retry(chain, content, max_retries=5):
    async with sem:
        for i in range(max_retries):
            try:
                return await asyncio.wait_for(chain.ainvoke({"benefit_content": content}), timeout=60.0)
            except Exception as e:
                if "rate_limit" in str(e).lower() or "429" in str(e):
                    wait_time = (2 ** (i + 1)) + random.uniform(0, 1)
                    await asyncio.sleep(wait_time)
                else: raise e
        return None

# 6. 배치 처리 함수
async def process_batch(batch_chunks, all_data):
    chain = prompt | llm
    tasks = [invoke_with_retry(chain, b['content']) for b in batch_chunks]
    responses = await asyncio.gather(*tasks, return_exceptions=True)
    
    batch_results = []
    for idx, res in enumerate(responses):
        if res is None or isinstance(res, Exception): continue
        try:
            raw_content = res.content.strip().replace('```json', '').replace('```', '')
            ai_data = json.loads(raw_content)
            meta = batch_chunks[idx]['metadata']
            
            # 오답(Negative) 구성
            same_card_others = [
                item['content'] for item in all_data 
                if item['metadata']['card_id'] == meta['card_id'] and item['metadata']['category'] != meta['category']
            ]
            raw_neg = random.choice(same_card_others) if same_card_others else "관련 혜택 없음"
            negative_text = raw_neg[:300] + "..." if len(raw_neg) > 300 else raw_neg

            batch_results.append({
                "anchor": ai_data['anchor'],
                "positive": ai_data['positive'],
                "negative": negative_text,
                "metadata": {
                    "card_name": meta['card_name'],
                    "category": meta['category'],
                    "card_id": meta['card_id'],
                    "card_type": meta.get('card_type'), # 신용/체크 구분 저장
                    "corp": meta.get('corp', '알 수 없음')
                }
            })
        except: continue
    return batch_results

# 7. 메인 실행 함수
async def main():
    # 데이터 로드 시 타입 태그 부여
    input_files = [
        {'path': 'all_credit_benefits_rag.json', 'type': 'credit'},
        {'path': 'all_check_benefits_rag.json', 'type': 'check'}
    ]
    
    all_benefits = []
    for f_info in input_files:
        if os.path.exists(f_info['path']):
            with open(f_info['path'], 'r', encoding='utf-8') as f:
                data = json.load(f)
                for item in data:
                    item['metadata']['card_type'] = f_info['type']
                all_benefits.extend(data)

    random.shuffle(all_benefits)

    # 이어하기 체크
    final_dataset = []
    if os.path.exists(OUTPUT_FILE):
        with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
            final_dataset = json.load(f)
            print(f"🔄 기존 데이터 {len(final_dataset)}건 발견. 재개합니다.")

    processed_indices = len(final_dataset)
    batch_size = 20 

    for i in range(processed_indices, len(all_benefits), batch_size):
        batch = all_benefits[i:i+batch_size]
        try:
            results = await asyncio.wait_for(process_batch(batch, all_benefits), timeout=180.0)
            final_dataset.extend(results)
            
            with open(OUTPUT_FILE, 'w', encoding='utf-8') as out:
                json.dump(final_dataset, out, ensure_ascii=False, indent=2)
            
            print(f"💾 {min(i + batch_size, len(all_benefits))} / {len(all_benefits)} 완료")
            await asyncio.sleep(0.5) 
        except Exception as e:
            print(f"❌ 에러: {e}. 5초 후 재개")
            await asyncio.sleep(5)
            continue

    print(f"✨ 전체 작업 완료! 파일: {OUTPUT_FILE}")

if __name__ == "__main__":
    await main()

💾 20 / 7763 완료
💾 40 / 7763 완료
💾 60 / 7763 완료
💾 80 / 7763 완료
💾 100 / 7763 완료
💾 120 / 7763 완료
💾 140 / 7763 완료
💾 160 / 7763 완료
💾 180 / 7763 완료
💾 200 / 7763 완료
💾 220 / 7763 완료
💾 240 / 7763 완료
💾 260 / 7763 완료
💾 280 / 7763 완료
💾 300 / 7763 완료
💾 320 / 7763 완료
💾 340 / 7763 완료
💾 360 / 7763 완료
💾 380 / 7763 완료
💾 400 / 7763 완료
💾 420 / 7763 완료
💾 440 / 7763 완료
💾 460 / 7763 완료
💾 480 / 7763 완료
💾 500 / 7763 완료
💾 520 / 7763 완료
💾 540 / 7763 완료
💾 560 / 7763 완료
💾 580 / 7763 완료
💾 600 / 7763 완료
💾 620 / 7763 완료
💾 640 / 7763 완료
💾 660 / 7763 완료
💾 680 / 7763 완료
💾 700 / 7763 완료
💾 720 / 7763 완료
💾 740 / 7763 완료
💾 760 / 7763 완료
💾 780 / 7763 완료
💾 800 / 7763 완료
💾 820 / 7763 완료
💾 840 / 7763 완료
💾 860 / 7763 완료
💾 880 / 7763 완료
💾 900 / 7763 완료
💾 920 / 7763 완료
💾 940 / 7763 완료
💾 960 / 7763 완료
💾 980 / 7763 완료
💾 1000 / 7763 완료
💾 1020 / 7763 완료
💾 1040 / 7763 완료
💾 1060 / 7763 완료
💾 1080 / 7763 완료
💾 1100 / 7763 완료
💾 1120 / 7763 완료
💾 1140 / 7763 완료
💾 1160 / 7763 완료
💾 1180 / 7763 완료
💾 1200 / 7763 완료
💾 1220 / 7763 완료
💾 1240 / 7763 완료

In [5]:
import json
import os
from collections import Counter

# 검사할 파일명 (본인의 파일명으로 수정하세요)
target_file = 'final_compact_dataset4.json'

def run_integrity_check(file_path):
    if not os.path.exists(file_path):
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return

    with open(file_path, 'r', encoding='utf-8') as f:
        dataset = json.load(f)

    # 1. 변수 초기화
    total = len(dataset)
    empty_anchor = 0
    empty_positive = 0
    anchor_lens = []
    positive_lens = []
    anchors = []
    corps = []
    categories = []

    # 2. 루프 돌며 데이터 수집
    for item in dataset:
        a = item.get('anchor', '').strip()
        p = item.get('positive', '').strip()
        m = item.get('metadata', {})
        
        if not a: empty_anchor += 1
        if not p: empty_positive += 1
        
        anchor_lens.append(len(a))
        positive_lens.append(len(p))
        anchors.append(a)
        corps.append(m.get('corp', 'Unknown'))
        categories.append(m.get('category', 'Unknown'))

    # 3. 통계 계산
    unique_anchors_count = len(set(anchors))
    duplicate_count = total - unique_anchors_count
    avg_anchor_len = sum(anchor_lens) / total if total > 0 else 0
    corp_dist = Counter(corps)
    cat_dist = Counter(categories)

    # 4. 결과 출력
    print(f"✅ 검사 완료: {file_path}")
    print("="*50)
    print(f"📊 [기본 통계]")
    print(f"· 총 데이터 개수: {total}건")
    print(f"· 중복된 질문(Anchor): {duplicate_count}건")
    print(f"· 질문 평균 길이: {avg_anchor_len:.1f}자")
    print(f"· 질문 최소/최대 길이: {min(anchor_lens)}자 / {max(anchor_lens)}자")
    print("-" * 50)
    
    print(f"🔍 [데이터 결함]")
    print(f"· 질문(Anchor) 누락: {empty_anchor}건")
    print(f"· 정답(Positive) 누락: {empty_positive}건")
    print("-" * 50)
    
    print(f"🏢 [카드사별 분포 (Top 5)]")
    for corp, count in corp_dist.most_common(5):
        print(f"· {corp}: {count}건 ({count/total*100:.1f}%)")
    print("-" * 50)
    
    print(f"🏷️ [카테고리별 분포 (Top 5)]")
    for cat, count in cat_dist.most_common(5):
        print(f"· {cat}: {count}건")
    print("="*50)

# 실행
run_integrity_check(target_file)

✅ 검사 완료: final_compact_dataset4.json
📊 [기본 통계]
· 총 데이터 개수: 7763건
· 중복된 질문(Anchor): 0건
· 질문 평균 길이: 79.0자
· 질문 최소/최대 길이: 51자 / 124자
--------------------------------------------------
🔍 [데이터 결함]
· 질문(Anchor) 누락: 0건
· 정답(Positive) 누락: 0건
--------------------------------------------------
🏢 [카드사별 분포 (Top 5)]
· KB국민카드: 1168건 (15.0%)
· 신한카드: 983건 (12.7%)
· 우리카드: 852건 (11.0%)
· NH농협카드: 631건 (8.1%)
· 롯데카드: 606건 (7.8%)
--------------------------------------------------
🏷️ [카테고리별 분포 (Top 5)]
· 유의사항: 1416건
· 카페: 307건
· 적립: 297건
· 기타: 290건
· 생활: 267건


In [ ]:
import json
import random
from sklearn.model_selection import train_test_split

def prepare_dl_dataset(credit_file, check_file):
    # 1. 각각의 파일 로드
    with open(credit_file, 'r', encoding='utf-8') as f:
        credit_data = json.load(f)
    with open(check_file, 'r', encoding='utf-8') as f:
        check_data = json.load(f)
    
    print(f"✅ 데이터 로드 완료: 신용({len(credit_data)}건), 체크({len(check_data)}건)")

    # 2. 통합 및 셔플링
    full_dataset = credit_data + check_data
    random.shuffle(full_dataset) # 신용/체크가 골고루 섞이도록 셔플
    
    print(f"🔄 전체 데이터 통합 완료: 총 {len(full_dataset)}건")

    # 3. 데이터 분할 (8:1:1 비율)
    # Train: 80%, Validation: 10%, Test: 10%
    train_data, temp_data = train_test_split(full_dataset, test_size=0.2, random_state=42)
    val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

    # 4. 각 세트 저장
    datasets = {
        'train_dataset.json': train_data,
        'val_dataset.json': val_data,
        'test_dataset.json': test_data
    }

    for filename, data in datasets.items():
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"💾 {filename} 저장 완료 ({len(data)}건)")

    return test_data # 성능 검증을 위해 test_data 반환

# 실행
test_set = prepare_dl_dataset('final_credit_dataset.json', 'final_check_dataset.json')